<a href="https://colab.research.google.com/github/GeeteshT/Project/blob/master/Bank_Note_Authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/gdrive")

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df=pd.read_csv(r"/gdrive/My Drive/Project/BankNote_Authentication.csv")

In [4]:
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [5]:
#Independent And Depend varaible
X=df.iloc[:,:-1]
Y=df.iloc[:,-1]

In [6]:
X.head()

,variance,skewness,curtosis,entropy
0,3.62160,8.6661,-2.8073,-0.44699
1,4.54590,8.1674,-2.4586,-1.46210
2,3.86600,-2.6383,1.9242,0.10645
3,3.45660,9.5228,-4.0112,-3.59440
4,0.32924,-4.4552,4.5718,-0.98880


In [7]:
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [10]:
from sklearn.ensemble import RandomForestClassifier

classifier= RandomForestClassifier()

classifier.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
y_pred= classifier.predict(x_test)

In [12]:
pd.DataFrame(list(zip(y_test,y_pred)),columns=["Real","Predicted"])

,Real,Predicted
0,1,1
1,0,0
2,1,1
3,0,0
4,0,0
...,...,...
270,1,1
271,0,0
272,1,1
273,1,1


In [13]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [14]:
roc_auc_score(y_train,classifier.predict(x_train))

1.0

In [15]:
roc_auc_score(y_test,classifier.predict(x_test))

0.9851559969772212

In [16]:
# Creating pickle file 

import pickle
pickle_out= open("classifier.pkl","wb") #wb=write byte mode
pickle.dump(classifier,pickle_out)
pickle_out.close()

In [17]:
classifier.predict([[2,3,4,1]])

array([0])

In [18]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://rm2il2x1fjf-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [19]:
from flask import Flask,request
import pickle 
test=pd.read_csv(r"/gdrive/My Drive/Project/Bank_Note_Authentication_test - Sheet1.csv")
test.head(2)

,variance,skewness,curtosis,entropy
0,3.6216,8.6661,-2.8073,-0.44699
1,4.5459,8.1674,-2.4586,-1.46210


In [21]:
#from flask import Flask,request

app= Flask(__name__)

pickle_in=open("classifier.pkl","rb")

classifier= pickle.load(pickle_in)

@app.route("/") #decorated
def welcome():
  return "hello Geetesh"


@app.route("/predict")
def predict_note_authentication():
  variance= request.args.get("variance")
  skewness= request.args.get("skewness")
  curtosis= request.args.get("curtosis")
  entropy= request.args.get("entropy")
  prediction = classifier.predict([[variance,skewness,curtosis,entropy]])
  return "The Predicted value is "+ str(prediction)

@app.route("/predict_file",methods=["POST"])
def predict_note_file():
  df_test= pd.read_csv(request.files.get("Bank_Note_Authentication"))
  prediction= classifier.predict(df_test)     
  return "The Predicted value is "+ str(prediction)


if __name__=="__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2020 04:20:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2020 04:20:06] "GET /favicon.ico HTTP/1.1" 404 -
[2020-10-24 04:20:16,217] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936,